In [12]:
import requests
import os
import json
import numpy as np
import pandas as pd
import tweepy

In [57]:
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAAYybAEAAAAA0hmnUdCAmXA3SywtTwmyGk%2BhquI%3DJ5BYZY2e4SHf2yWdMQF483ZPz4sOb8t78akzp1PjfLNO2NGY3A')

# Replace with your own search query
#query = 'from:petrogustavo -is:retweet'
#query = 'Petro'
query = 'from:petrogustavo'
tweet_fields=['context_annotations', 'created_at', 'author_id', 'text', 'public_metrics']

tweets = client.search_recent_tweets(query=query, tweet_fields=tweet_fields, max_results=100)

tweet_df = pd.DataFrame(data=[tweet.text for tweet in tweets.data], columns=['Texto'])

In [28]:
#data = []
#columns = ['Fecha', 'Texto', 'Metricas']

#for tweet in tweets.data:
#    data.append([tweet.created_at, tweet.text, tweet.public_metrics])
#df = pd.DataFrame(data, columns=columns)
#df.head()

In [40]:
print(dir(tweets.data[0]))

['__abstractmethods__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_abc_impl', 'attachments', 'author_id', 'context_annotations', 'conversation_id', 'created_at', 'data', 'entities', 'geo', 'get', 'id', 'in_reply_to_user_id', 'items', 'keys', 'lang', 'non_public_metrics', 'organic_metrics', 'possibly_sensitive', 'promoted_metrics', 'public_metrics', 'referenced_tweets', 'reply_settings', 'source', 'text', 'values', 'withheld']


In [58]:
# We add relevant data:
tweet_df['id']   = np.array([tweet.id for tweet in tweets.data])
tweet_df['author_id']   = np.array([tweet.author_id for tweet in tweets.data])
tweet_df['len']  = np.array([len(tweet.text) for tweet in tweets.data])
tweet_df['Date'] = np.array([tweet.created_at for tweet in tweets.data])
tweet_df['Geo'] = pd.Series([tweet.geo for tweet in tweets.data])
tweet_df['Source'] = np.array([tweet.source for tweet in tweets.data])
tweet_df['public_metrics']  = np.array([tweet.public_metrics for tweet in tweets.data])
tweet_df['non_public_metrics']  = np.array([tweet.non_public_metrics for tweet in tweets.data])
tweet_df['context_annotations']    = np.array([tweet.context_annotations for tweet in tweets.data])
tweet_df['values']    = np.array([tweet.values for tweet in tweets.data])
tweet_df['keys']    = np.array([tweet.keys for tweet in tweets.data])
tweet_df['attachments']=  np.array([tweet.attachments for tweet in tweets.data])

#tweet_df['username']=  np.array([tweet.user_id for tweet in tweets.data])
#tweet_df['public_metrics']  = tweet_df.public_metrics.to_dict()
#tweet_df['like_count'] = tweet_df.public_metrics.like_count

/var/folders/_w/n5m2rv0x26lc28m7jmscffwjbkmphb/T/ipykernel_7111/439503068.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tweet_df['context_annotations']    = np.array([tweet.context_annotations for tweet in tweets.data])


In [60]:
tweet_df['pos'] = tweet_df['Texto'].str.find('@')
tweet_df['pos2'] = tweet_df['Texto'].str.find(':')
tweet_df['Username'] = tweet_df.apply(lambda x: x['Texto'][x['pos']:x['pos2']],axis=1)

tweet_df['tmp_rt'] = tweet_df['Texto'].str[:2]
tweet_df['RT'] = np.where(tweet_df['tmp_rt']== 'RT', True, False)
tweet_df = tweet_df.drop(columns=['pos', 'pos2', 'tmp_rt'])

#tweet_df

In [61]:
tweet_df.info()
tweet_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   Texto                100 non-null    object             
 1   id                   100 non-null    int64              
 2   author_id            100 non-null    int64              
 3   len                  100 non-null    int64              
 4   Date                 100 non-null    datetime64[ns, UTC]
 5   Geo                  0 non-null      object             
 6   Source               0 non-null      object             
 7   public_metrics       100 non-null    object             
 8   non_public_metrics   0 non-null      object             
 9   context_annotations  100 non-null    object             
 10  values               100 non-null    object             
 11  keys                 100 non-null    object             
 12  attachments          0 

,Texto,id,author_id,len,Date,Geo,Source,public_metrics,non_public_metrics,context_annotations,values,keys,attachments,Username,RT
0,La exportación de carne solo debe ser posible ...,1512812483823558656,49849732,84,2022-04-09 15:19:37+00:00,None,None,"{'retweet_count': 62, 'reply_count': 14, 'like...",None,"[{'domain': {'id': '10', 'name': 'Person', 'de...",<bound method Mapping.values of <Tweet id=1512...,<bound method Mapping.keys of <Tweet id=151281...,None,,False
1,Hoy la carne en Colombia vale en dólares el do...,1512812481344905236,49849732,279,2022-04-09 15:19:36+00:00,None,None,"{'retweet_count': 82, 'reply_count': 25, 'like...",None,"[{'domain': {'id': '10', 'name': 'Person', 'de...",<bound method Mapping.values of <Tweet id=1512...,<bound method Mapping.keys of <Tweet id=151281...,None,,False
2,RT @anatolykarpovv: Aquí Rodolfo Hernández @ln...,1512810519723454464,49849732,140,2022-04-09 15:11:49+00:00,None,None,"{'retweet_count': 239, 'reply_count': 0, 'like...",None,"[{'domain': {'id': '10', 'name': 'Person', 'de...",<bound method Mapping.values of <Tweet id=1512...,<bound method Mapping.keys of <Tweet id=151281...,None,@anatolykarpovv,True
3,"Dos alcaldes de Bogotá. Gaitán y Nariño, son l...",1512809197490089987,49849732,293,2022-04-09 15:06:33+00:00,None,None,"{'retweet_count': 205, 'reply_count': 73, 'lik...",None,"[{'domain': {'id': '10', 'name': 'Person', 'de...",<bound method Mapping.values of <Tweet id=1512...,<bound method Mapping.keys of <Tweet id=151280...,None,,False
4,La destrucción de la política de paz hecha por...,1512785883115757569,49849732,285,2022-04-09 13:33:55+00:00,None,None,"{'retweet_count': 391, 'reply_count': 108, 'li...",None,"[{'domain': {'id': '10', 'name': 'Person', 'de...",<bound method Mapping.values of <Tweet id=1512...,<bound method Mapping.keys of <Tweet id=151278...,None,,False
5,El gobierno de Duque permitió que el ELN dupli...,1512784567593275392,49849732,258,2022-04-09 13:28:41+00:00,None,None,"{'retweet_count': 827, 'reply_count': 177, 'li...",None,"[{'domain': {'id': '10', 'name': 'Person', 'de...",<bound method Mapping.values of <Tweet id=1512...,<bound method Mapping.keys of <Tweet id=151278...,None,,False
6,Duque expandió el Clan del Golfo. Le incumplió...,1512783425245257737,49849732,296,2022-04-09 13:24:09+00:00,None,None,"{'retweet_count': 474, 'reply_count': 73, 'lik...",None,"[{'domain': {'id': '10', 'name': 'Person', 'de...",<bound method Mapping.values of <Tweet id=1512...,<bound method Mapping.keys of <Tweet id=151278...,None,,False
7,Esta expansión de las disidencias que entrega ...,1512781341640867840,49849732,300,2022-04-09 13:15:52+00:00,None,None,"{'retweet_count': 414, 'reply_count': 90, 'lik...",None,"[{'domain': {'id': '10', 'name': 'Person', 'de...",<bound method Mapping.values of <Tweet id=1512...,<bound method Mapping.keys of <Tweet id=151278...,None,,False
8,RT @estebanalitic: En el reciente informe pres...,1512780653477179397,49849732,140,2022-04-09 13:13:08+00:00,None,None,"{'retweet_count': 1186, 'reply_count': 0, 'lik...",None,"[{'domain': {'id': '10', 'name': 'Person', 'de...",<bound method Mapping.values of <Tweet id=1512...,<bound method Mapping.keys of <Tweet id=151278...,None,@estebanalitic,True
9,"RT @tavo2366: En contravención a la ley, el se...",1512780444462366720,49849732,140,2022-04-09 13:12:18+00:00,None,None,"{'retweet_count': 959, 'reply_count': 0, 'like...",None,"[{'domain': {'id': '10', 'name': 'Person', 'de...",<bound method Mapping.values of <Tweet id=1512...,<bound method Mapping.keys of <Tweet id=151278...,None,@tavo2366,True


In [21]:
tweet_df.Date.describe()


/var/folders/_w/n5m2rv0x26lc28m7jmscffwjbkmphb/T/ipykernel_7111/277588426.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  tweet_df.Date.describe()


count                           100
unique                           40
top       2022-04-09 14:41:23+00:00
freq                              5
first     2022-04-09 14:41:22+00:00
last      2022-04-09 14:42:08+00:00
Name: Date, dtype: object

In [26]:
import requests
import os
import json

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get("BEARER_TOKEN")
bearer_token='AAAAAAAAAAAAAAAAAAAAAAYybAEAAAAA0hmnUdCAmXA3SywtTwmyGk%2BhquI%3DJ5BYZY2e4SHf2yWdMQF483ZPz4sOb8t78akzp1PjfLNO2NGY3A'

search_url = "https://api.twitter.com/2/tweets/search/recent"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    json_response = connect_to_endpoint(search_url, query_params)
    #print(json.dumps(json_response, indent=4, sort_keys=True))
    #j = json_response.json()
    df = pd.DataFrame(json_response)
    return df
    
if __name__ == "__main__":
    main()



200


ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.